<a href="https://colab.research.google.com/github/Soorajb19/Scene_Recognition/blob/main/AMLFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division

import torch

import torch.nn as nn

import torch.optim as optim

from torch.optim import lr_scheduler

import numpy as np

import torchvision

from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt

import time

import os

import copy



In [ ]:
from time import time
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/AMLME/Places2_simp.zip

Streaming output truncated to the last 5000 lines.
  inflating: Places2_simp/phone_booth/00000414.jpg  
  inflating: Places2_simp/phone_booth/00000399.jpg  
  inflating: Places2_simp/phone_booth/00000158.jpg  
  inflating: Places2_simp/phone_booth/00000164.jpg  
  inflating: Places2_simp/phone_booth/00000602.jpg  
  inflating: Places2_simp/phone_booth/00000616.jpg  
  inflating: Places2_simp/phone_booth/00000170.jpg  
  inflating: Places2_simp/phone_booth/00000825.jpg  
  inflating: Places2_simp/phone_booth/00000831.jpg  
  inflating: Places2_simp/phone_booth/00000819.jpg  
  inflating: Places2_simp/phone_booth/00000992.jpg  
  inflating: Places2_simp/phone_booth/00000986.jpg  
  inflating: Places2_simp/phone_booth/00000038.jpg  
  inflating: Places2_simp/phone_booth/00000010.jpg  
  inflating: Places2_simp/phone_booth/00000776.jpg  
  inflating: Places2_simp/phone_booth/00000762.jpg  
  inflating: Places2_simp/phone_booth/00000004.jpg  
  inflating: Places2_simp/phone_booth/00000789.j

In [ ]:
! unzip /content/drive/MyDrive/AMLME/test.zip

Archive:  /content/drive/MyDrive/AMLME/test.zip
   creating: test/
   creating: test/airport_terminal/
  inflating: test/airport_terminal/2000x1125-terminal-5-t5-heathrow-airportc-david-j-osborn.jpg  
  inflating: test/airport_terminal/28f195d0-8f10-11e8-b6b0-0242ac110007.jpg  
  inflating: test/airport_terminal/2_JS264501658.jpg  
  inflating: test/airport_terminal/8eec424e19869185009a80261e583b33Y29udGVudHNlYXJjaGFwaSwxNjc2MzYxODk0-2.70430304.jpg  
  inflating: test/airport_terminal/blog-heathrow1_Belinda_Fewings_for_Unsplash.jpg  
   creating: test/amphitheatre/
  inflating: test/amphitheatre/aerial-view-of-the-al-dana-amphitheatre-in-bahrain-showing-stage-seating-and-entrance-circulation.jpg  
  inflating: test/amphitheatre/Arles_-_2017-05-24_-_Roman_Amphitheatre_-_3804.jpg  
  inflating: test/amphitheatre/Gallo_Roman_Amphitheater.jpg  
  inflating: test/amphitheatre/JS119350861_ANSA_The-Verona-Arena_trans_NvBQzQNjv4BqUKDEcAaGeXPwThJmcFHx4bACH36HgLKbf4uGDnMZpuE.jpg  
  inflating: t

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_data = datasets.ImageFolder('/content/Places2_simp', transform=transform)
val_data = datasets.ImageFolder('/content/test', transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=True)

In [ ]:
import glob
target_names = [st.split("/")[-1] for st in glob.glob("/content/Places2_simp/*")]
target_names.sort()
target_names

['airport_terminal',
 'amphitheatre',
 'amusement_park',
 'art_gallery',
 'bakery_shop',
 'bar',
 'bookstore',
 'botanical_garden',
 'bridge',
 'bus_interior',
 'butchers_shop',
 'campsite',
 'classroom',
 'coffee_shop',
 'construction_site',
 'courtyard',
 'driveway',
 'fire_station',
 'fountain',
 'gas_station',
 'harbour',
 'highway',
 'kindergarten_classroom',
 'lobby',
 'market_outdoor',
 'museum',
 'office',
 'parking_lot',
 'phone_booth',
 'playground',
 'railroad_track',
 'restaurant',
 'river',
 'shed',
 'staircase',
 'supermarket',
 'swimming_pool_outdoor',
 'track',
 'valley',
 'yard']

In [ ]:
target_2_int = {}
counter = 0
for _ in target_names:
    target_2_int[_] = counter
    counter += 1

target_2_int

{'airport_terminal': 0,
 'amphitheatre': 1,
 'amusement_park': 2,
 'art_gallery': 3,
 'bakery_shop': 4,
 'bar': 5,
 'bookstore': 6,
 'botanical_garden': 7,
 'bridge': 8,
 'bus_interior': 9,
 'butchers_shop': 10,
 'campsite': 11,
 'classroom': 12,
 'coffee_shop': 13,
 'construction_site': 14,
 'courtyard': 15,
 'driveway': 16,
 'fire_station': 17,
 'fountain': 18,
 'gas_station': 19,
 'harbour': 20,
 'highway': 21,
 'kindergarten_classroom': 22,
 'lobby': 23,
 'market_outdoor': 24,
 'museum': 25,
 'office': 26,
 'parking_lot': 27,
 'phone_booth': 28,
 'playground': 29,
 'railroad_track': 30,
 'restaurant': 31,
 'river': 32,
 'shed': 33,
 'staircase': 34,
 'supermarket': 35,
 'swimming_pool_outdoor': 36,
 'track': 37,
 'valley': 38,
 'yard': 39}

In [ ]:
model = models.resnet34(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 331MB/s]


In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(in_features=512, out_features=40)

In [ ]:
from torch.optim import Adam
loss_fn = nn.CrossEntropyLoss()
optimiser = Adam(model.parameters(), lr=1e-2, weight_decay=0.0001)
model = model.to(device)


In [ ]:
LEN_TRAIN = len(train_data)
LEN_TEST = len(val_data)

In [ ]:

for epoch in range(20):
    start = time()

    tr_acc = 0
    test_acc = 0

    # Train
    model.train()

    with tqdm(train_loader, unit="batch") as tepoch:
        for xtrain, ytrain in tepoch:
            optimiser.zero_grad()

            xtrain = xtrain.to(device)
            train_prob = model(xtrain)
            train_prob = train_prob.cpu()

            loss = loss_fn(train_prob, ytrain)
            loss.backward()
            optimiser.step()

            # training ends

            train_pred = torch.max(train_prob, 1).indices
            tr_acc += int(torch.sum(train_pred == ytrain))

        ep_tr_acc = tr_acc / LEN_TRAIN

    # Evaluate
    model.eval()
    with torch.no_grad():
        for xtest, ytest in test_loader:
            xtest = xtest.to(device)
            test_prob = model(xtest)
            test_prob = test_prob.cpu()

            test_pred = torch.max(test_prob,1).indices
            test_acc += int(torch.sum(test_pred == ytest))

        ep_test_acc = test_acc / LEN_TEST

    end = time()
    duration = (end - start) / 60

    print(f"Epoch: {epoch}, Time: {duration}, Loss: {loss}\nTrain_acc: {ep_tr_acc}, Test_acc: {ep_test_acc}")

100%|██████████| 625/625 [03:18<00:00,  3.15batch/s]


Epoch: 0, Time: 3.5048345923423767, Loss: 0.38042259216308594
Train_acc: 0.889425, Test_acc: 0.5050505050505051


100%|██████████| 625/625 [03:19<00:00,  3.14batch/s]


Epoch: 1, Time: 3.5206682960192364, Loss: 0.06848844885826111
Train_acc: 0.975625, Test_acc: 0.4797979797979798


100%|██████████| 625/625 [03:18<00:00,  3.15batch/s]


...

Epoch: 17, Time: 3.522047734260559, Loss: 0.11752715706825256
Train_acc: 0.9878, Test_acc: 0.47474747474747475


100%|██████████| 625/625 [03:18<00:00,  3.15batch/s]


Epoch: 18, Time: 3.5133110920588178, Loss: 0.036430034786462784
Train_acc: 0.991875, Test_acc: 0.51010101010101


100%|██████████| 625/625 [03:18<00:00,  3.15batch/s]


Epoch: 19, Time: 3.511802156766256, Loss: 0.06679725646972656
Train_acc: 0.990725, Test_acc: 0.5454545454545454


In [ ]:
def top_k_acc(k):

  top_5_correct = 0
  total_samples = 0

  for i, (X, y) in enumerate(test_loader):
    X = X.to(device)
    y = y.to(device)
    model.eval()
    pred = model(X)

    _, predicted = torch.topk(pred, k=k, dim=1)
    # Check if the true labels are in the top 5 predictions
    top_5_correct += (predicted == y.view(-1, 1)).sum().item()
    total_samples += y.size(0)

  top_5_accuracy = (top_5_correct / total_samples) * 100
  return (top_5_accuracy)

In [ ]:
print('Top 1 accuracy:' , top_k_acc(1))
print('Top 2 accuracy:' , top_k_acc(2))
print('Top 5 accuracy:' , top_k_acc(5))
print('Top 10 accuracy:' , top_k_acc(10))

Top 1 accuracy: 54.54545454545454
Top 2 accuracy: 68.68686868686868
Top 5 accuracy: 83.33333333333334
Top 10 accuracy: 90.9090909090909
